In [1]:
import sys
sys.path.insert(0, 'C:\\Users\\masch\\QuantumComputing\\QComp\\pgmpy')

# Imports
import cmath
import math
import numpy as np
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete.CPD import TabularCPD
from pgmpy.inference import VariableElimination
from pgmpy.inference import BeliefPropagation
from pgmpy.readwrite import BIFReader, BIFWriter


bellState = BayesianNetwork([('q0m0', 'q0m1'), ('q0m1', 'q0m2'), ('q1m0', 'q1m1'), ('q1m1', 'q1m2'), ('q0m1', 'q1m2')])

cpd_q0m0 = TabularCPD(variable = 'q0m0', variable_card = 2, values = [[1], [0]])
cpd_q1m0 = TabularCPD(variable = 'q1m0', variable_card = 2, values = [[1], [0]])
cpd_q0m1 = TabularCPD(variable='q0m1', variable_card = 2, values = [[1/np.sqrt(2), 1/np.sqrt(2)], [1/np.sqrt(2), -1/np.sqrt(2)]], evidence = ['q0m0'], evidence_card = [2])
cpd_q1m1 = TabularCPD(variable = 'q1m1', variable_card = 2, values = [[1, 0], [0,1]], evidence = ['q1m0'], evidence_card = [2])
cpd_q1m2 = TabularCPD(variable='q1m2', variable_card = 2, values = [[1,0,0,1], [0,1,1,0]], evidence = ['q0m1', 'q1m1'], evidence_card = [2,2])
cpd_q0m2 = TabularCPD(variable='q0m2', variable_card = 2, values = [[1, 0], [0, 1]], evidence = ['q0m1'], evidence_card = [2])

bellState.add_cpds(cpd_q0m0, cpd_q1m0, cpd_q0m1, cpd_q0m2, cpd_q1m2, cpd_q1m1)
bellStateInfer = VariableElimination(bellState)
q1 = bellStateInfer.query(['q1m2', 'q0m2'])

print(q1)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

+---------+---------+------------------+
| q0m2    | q1m2    |   phi(q0m2,q1m2) |
+=========+=========+==================+
| q0m2(0) | q1m2(0) |   0.7071+0.0000j |
+---------+---------+------------------+
| q0m2(0) | q1m2(1) |   0.0000+0.0000j |
+---------+---------+------------------+
| q0m2(1) | q1m2(0) |   0.0000+0.0000j |
+---------+---------+------------------+
| q0m2(1) | q1m2(1) |   0.7071+0.0000j |
+---------+---------+------------------+


C:\Users\masch\QuantumComputing\QComp\pgmpy\pgmpy\extern\tabulate.py:282: ComplexWarning: Casting complex values to real discards the imaginary part
  n = conv(string)


In [2]:
writer = BIFWriter(bellState)
writer.write_bif(filename='bellState.bif')

In [3]:
reader = BIFReader("bellState.bif")
reader.get_values()

{'q0m0': array([[1.+0.j],
        [0.+0.j]]),
 'q0m1': array([[ 0.70710678+0.j,  0.70710678+0.j],
        [ 0.70710678+0.j, -0.70710678+0.j]]),
 'q0m2': array([[1.+0.j, 0.+0.j],
        [0.+0.j, 1.+0.j]]),
 'q1m0': array([[1.+0.j],
        [0.+0.j]]),
 'q1m1': array([[1.+0.j, 0.+0.j],
        [0.+0.j, 1.+0.j]]),
 'q1m2': array([[1.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
        [0.+0.j, 1.+0.j, 1.+0.j, 0.+0.j]])}

In [4]:
bellState2 = reader.get_model()

In [5]:
bellState2Infer = VariableElimination(bellState2)
q2 = bellState2Infer.query(['q1m2', 'q0m2'])

print(q2)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

+---------+---------+------------------+
| q0m2    | q1m2    |   phi(q0m2,q1m2) |
+=========+=========+==================+
| q0m2(0) | q1m2(0) |   0.7071+0.0000j |
+---------+---------+------------------+
| q0m2(0) | q1m2(1) |   0.0000+0.0000j |
+---------+---------+------------------+
| q0m2(1) | q1m2(0) |   0.0000+0.0000j |
+---------+---------+------------------+
| q0m2(1) | q1m2(1) |   0.7071+0.0000j |
+---------+---------+------------------+


In [6]:
pA = 0.1
pB = 0.1
pB_givenA = 0.3
pA_givenB = pB_givenA*pA/pB
pB_givenNotA = (1-pA_givenB)*pB/(1-pA)
#p(B|!A) = p(B,!A)/p(!A) = p(!A|B)*p(B)/p(!A) = (1-p(A|B))*p(B)/(1-p(A)) = (p(B)-p(A,B))/(1-p(A))

corr_bit_flip = BayesianNetwork([('q0m0','q0m1'),('q1m0','q1m1'),('rv1','rv2'),('rv1','q0m1'),('rv2','q1m1')])
cpd_q0m0 = TabularCPD(variable='q0m0',variable_card=2,values=[[1],[0]])
cpd_q1m0 = TabularCPD(variable='q1m0',variable_card=2,values=[[1],[0]])
cpd_rv1 = TabularCPD(variable='rv1',variable_card=2,values=[[np.sqrt(1-pA)],[np.sqrt(pA)]])
cpd_rv2 = TabularCPD(variable='rv2',variable_card=2,values=[[np.sqrt(1-pB),np.sqrt(1-pB_givenA)],[np.sqrt(pB),np.sqrt(pB_givenA)]],evidence=['rv1'],evidence_card=[2])
cpd_q0m1 = TabularCPD(variable='q0m1',variable_card=2,values=[[1,0,0,1],[0,1,1,0]],evidence=['q0m0','rv1'],evidence_card=[2,2])
cpd_q1m1 = TabularCPD(variable='q1m1',variable_card=2,values=[[1,0,0,1],[0,1,1,0]],evidence=['q1m0','rv2'],evidence_card=[2,2])

corr_bit_flip.add_cpds(cpd_q0m0,cpd_q1m0,cpd_rv1,cpd_rv2,cpd_q0m1,cpd_q1m1)
corr_infer = VariableElimination(corr_bit_flip)
corr_query = corr_infer.query(['rv1','rv2','q0m1','q1m1'])
print(corr_query)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

+--------+---------+--------+---------+--------------------------+
| rv1    | q1m1    | rv2    | q0m1    |   phi(rv1,q1m1,rv2,q0m1) |
+========+=========+========+=========+==========================+
| rv1(0) | q1m1(0) | rv2(0) | q0m1(0) |           0.9000+0.0000j |
+--------+---------+--------+---------+--------------------------+
| rv1(0) | q1m1(0) | rv2(0) | q0m1(1) |           0.0000+0.0000j |
+--------+---------+--------+---------+--------------------------+
| rv1(0) | q1m1(0) | rv2(1) | q0m1(0) |           0.0000+0.0000j |
+--------+---------+--------+---------+--------------------------+
| rv1(0) | q1m1(0) | rv2(1) | q0m1(1) |           0.0000+0.0000j |
+--------+---------+--------+---------+--------------------------+
| rv1(0) | q1m1(1) | rv2(0) | q0m1(0) |           0.0000+0.0000j |
+--------+---------+--------+---------+--------------------------+
| rv1(0) | q1m1(1) | rv2(0) | q0m1(1) |           0.0000+0.0000j |
+--------+---------+--------+---------+-----------------------

In [7]:
writer = BIFWriter(corr_bit_flip)
writer.write_bif(filename='corr_bit_flip.bif')
reader = BIFReader("corr_bit_flip.bif")
reader.get_values()

{'q0m0': array([[1.+0.j],
        [0.+0.j]]),
 'q0m1': array([[1.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
        [0.+0.j, 1.+0.j, 1.+0.j, 0.+0.j]]),
 'q1m0': array([[1.+0.j],
        [0.+0.j]]),
 'q1m1': array([[1.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
        [0.+0.j, 1.+0.j, 1.+0.j, 0.+0.j]]),
 'rv1': array([[0.9486833 +0.j],
        [0.31622777+0.j]]),
 'rv2': array([[0.9486833 +0.j, 0.83666003+0.j],
        [0.31622777+0.j, 0.54772256+0.j]])}

In [8]:
corr_bit_flip2 = reader.get_model()
corr_bit_flipInfer2 = VariableElimination(corr_bit_flip2)
corr_query2 = corr_bit_flipInfer2.query(['rv1','rv2','q0m1','q1m1'])
print(corr_query2)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

+--------+---------+--------+---------+--------------------------+
| rv1    | q1m1    | rv2    | q0m1    |   phi(rv1,q1m1,rv2,q0m1) |
+========+=========+========+=========+==========================+
| rv1(0) | q1m1(0) | rv2(0) | q0m1(0) |           0.9000+0.0000j |
+--------+---------+--------+---------+--------------------------+
| rv1(0) | q1m1(0) | rv2(0) | q0m1(1) |           0.0000+0.0000j |
+--------+---------+--------+---------+--------------------------+
| rv1(0) | q1m1(0) | rv2(1) | q0m1(0) |           0.0000+0.0000j |
+--------+---------+--------+---------+--------------------------+
| rv1(0) | q1m1(0) | rv2(1) | q0m1(1) |           0.0000+0.0000j |
+--------+---------+--------+---------+--------------------------+
| rv1(0) | q1m1(1) | rv2(0) | q0m1(0) |           0.0000+0.0000j |
+--------+---------+--------+---------+--------------------------+
| rv1(0) | q1m1(1) | rv2(0) | q0m1(1) |           0.0000+0.0000j |
+--------+---------+--------+---------+-----------------------